<b>

ISYS613 | Course Project | Fall 2021
--------|----------------|----------

</b>


## Project Overview
The objective of this project is to afford you the opportunity to demonstrate the
skills acquired over the course of the semester.  Specifically, you will be constructing
an analytics-ready dataset using data from several disparate, but related data sources.

Your dataset is to be a pandas dataframe suitable for answering the proof-of-concept
questions located at the end of this document.

### The Scenario
You are the proprietor of a small tour company.  You are interested in launching a new app
that will allow users to undertake self-guided tours of noteworthy authors. To launch your app
you need to assemble the authors' location information along with some other facts that you
believe will be of interest to your customers.

## Data Sources
You will be collecting and assembling data from several different sources and formats.
1. Source 1 - web scraping
1. Source 2 - delimitted file
1. Source 3 - direct download XML file
1. Source 4 - Zip Code API

## Project Teams
Working on a team is not a requirement and you may complete the project on your own.
Thus, project team(s) may be composed of 1 or **at most** 2 students. If you are working
with another student please, ensure both of your names are clearly visible in your final solution JNB.


### Source 1: Initial Author Data
The initial list of author names is to be web-scraped from a popular "famous quotes" website.
There are 50 unique authors available from this site.  Begin your exploration at the base
URL and determine your own strategy to find and capture the required author data.

Depending on your approach, you may encounter the same author multiple times.
In the end, your list of authors should be duplicate free.

```
Base Scrape URL: https://quotes.toscrape.com
```
#### Tasks
1. Beginning at the Base Scrape URL given above, you are to use a web scraping technique to collect
the following author information:
* Full Name
* Date of birth
* Birth location
2. Create a pandas dataframe from (or using) the relevant data

In [22]:
# TEST DATA
#URL = 'http://books.toscrape.com/'

from requests.exceptions import HTTPError
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from urllib.parse import urljoin

EW_URL = 'https://quotes.toscrape.com'

def simple_get(url, *args, **kwargs):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        resp = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        resp.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        raise http_err
    except Exception as err:
        print(f'Other error occurred: {err}')
        raise err

    return resp

allpage=[]      
def get_allurl2(url):
    resp = simple_get(url, timeout=5)
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser') 
    
    li = soup.find('li', class_='next')
    if li is not None:      
        pgurl = urljoin(url, li.a['href'])  
        allpage.append(pgurl)
        
        get_allurl2(pgurl)
    


def who_actors(allurl):
    
    author = []
    author_date = []
    author_location = []
    for url in allurl:
    
        resp = simple_get(url, timeout=5)
        html = resp.text

        soup = BeautifulSoup(html, 'html.parser')

        body_tag = soup.body


        tag = soup.find_all("div", class_="quote")

        for t in tag:
            a = t.find("small", class_="author")
            author.append(a.text)
            #print("author testing")
            #print(a.text)

            link = allurl[0]+ t.a["href"]
            #print(link)
            result = requests.get(link)
            soup_loop = BeautifulSoup(result.text, 'html.parser')
            auth_date = soup_loop.find("span", class_="author-born-date")
            author_date.append(auth_date.text)
            auth_location = soup_loop.find("span", class_="author-born-location")
            author_location.append(auth_location.text)

        #print(author)
        #print(author_date)
        #print(author_location)
    output = pd.DataFrame({'author': author, 'Author_Birth_Date': author_date,  "Author_Birth_Location": author_location})
    newoutput=output.drop_duplicates(subset='author', keep="last")
    newoutput.reset_index(drop=True)
    print(newoutput)

        #for p in soup.find_all("span", class_ = "author"):
            #print(p.text)

        #for img in soup.find_all('img', title=re.compile(r'^Slide\s+\d+:\s+[A-Z]')):
            #print("here is test2")
            #print(img)

        #print("here is testing")
        #print(soup.findAll("article", class_ = "product_pod"))

        #url_main_page = [EW_URL+x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]

    
def main():
    allpage.append(EW_URL)
    get_allurl2(EW_URL)
    #print(allpage)
    who_actors(allpage)
    
if __name__ == "__main__":
    main()

                    author   Author_Birth_Date  \
6               André Gide   November 22, 1869   
7         Thomas A. Edison   February 11, 1847   
9             Steve Martin     August 14, 1945   
15           Douglas Adams      March 11, 1952   
16             Elie Wiesel  September 30, 1928   
17     Friedrich Nietzsche    October 15, 1844   
19          Allen Saunders      April 24, 1899   
20            Pablo Neruda       July 12, 1904   
23        Garrison Keillor     August 07, 1942   
24              Jim Henson  September 24, 1936   
32              Bob Marley   February 06, 1945   
33           Mother Teresa     August 26, 1910   
35       Charles M. Schulz   November 26, 1922   
36       William Nicholson    January 12, 1948   
38       Jorge Luis Borges     August 24, 1899   
39            George Eliot   November 22, 1819   
47  Martin Luther King Jr.    January 15, 1929   
49           James Baldwin     August 02, 1924   
51       Eleanor Roosevelt    October 11, 1884   


### Source 2: Author Key Data
For each of the 50 authors previously identified, you are to merge a key and gender value available
from a CSV file with the author data from *Source 1*.

```
CSV File: *author_key_file.csv*
```
#### Tasks
1. The author names are unique in both
data sources and thus may be used to associate the *key* and *gender* attribute values with the author.
3. Once you have completed the merge, convert the *key* column to be the dataframe's row index.

In [7]:

# TEST DATA
#URL = 'http://books.toscrape.com/'

from requests.exceptions import HTTPError
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from urllib.parse import urljoin

EW_URL = 'https://quotes.toscrape.com'

def simple_get(url, *args, **kwargs):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        resp = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        resp.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        raise http_err
    except Exception as err:
        print(f'Other error occurred: {err}')
        raise err

    return resp

allpage=[]      
def get_allurl2(url):
    resp = simple_get(url, timeout=5)
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser') 
    
    li = soup.find('li', class_='next')
    if li is not None:      
        pgurl = urljoin(url, li.a['href'])  
        allpage.append(pgurl)
        
        get_allurl2(pgurl)
    


def who_actors(allurl):
    
    author = []
    author_date = []
    author_location = []
    for url in allurl:
    
        resp = simple_get(url, timeout=5)
        html = resp.text

        soup = BeautifulSoup(html, 'html.parser')

        body_tag = soup.body


        tag = soup.find_all("div", class_="quote")

        for t in tag:
            a = t.find("small", class_="author")
            author.append(a.text)
            #print("author testing")
            #print(a.text)

            link = allurl[0]+ t.a["href"]
            #print(link)
            result = requests.get(link)
            soup_loop = BeautifulSoup(result.text, 'html.parser')
            auth_date = soup_loop.find("span", class_="author-born-date")
            author_date.append(auth_date.text)
            auth_location = soup_loop.find("span", class_="author-born-location")
            author_location.append(auth_location.text)

        #print(author)
        #print(author_date)
        #print(author_location)
    output = pd.DataFrame({'author': author, 'Author_Birth_Date': author_date,  "Author_Birth_Location": author_location})
    newoutput=output.drop_duplicates(subset='author', keep="last")
    newoutput.reset_index(drop=True)
    print(newoutput)
    return newoutput

        #for p in soup.find_all("span", class_ = "author"):
            #print(p.text)

        #for img in soup.find_all('img', title=re.compile(r'^Slide\s+\d+:\s+[A-Z]')):
            #print("here is test2")
            #print(img)

        #print("here is testing")
        #print(soup.findAll("article", class_ = "product_pod"))

        #url_main_page = [EW_URL+x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]

    
def main():
    allpage.append(EW_URL)
    get_allurl2(EW_URL)
    #print(allpage)
    author_pd = who_actors(allpage)
    
    t  = pd.read_csv('author_key_file.csv', delimiter=",") 
    dfinal = author_pd.merge(t, how='left', left_on='author', right_on='name')
    
    dfinal.set_index('key', inplace = True)
    print(dfinal)
if __name__ == "__main__":
    main()

                    author   Author_Birth_Date  \
6               André Gide   November 22, 1869   
7         Thomas A. Edison   February 11, 1847   
9             Steve Martin     August 14, 1945   
15           Douglas Adams      March 11, 1952   
16             Elie Wiesel  September 30, 1928   
17     Friedrich Nietzsche    October 15, 1844   
19          Allen Saunders      April 24, 1899   
20            Pablo Neruda       July 12, 1904   
23        Garrison Keillor     August 07, 1942   
24              Jim Henson  September 24, 1936   
32              Bob Marley   February 06, 1945   
33           Mother Teresa     August 26, 1910   
35       Charles M. Schulz   November 26, 1922   
36       William Nicholson    January 12, 1948   
38       Jorge Luis Borges     August 24, 1899   
39            George Eliot   November 22, 1819   
47  Martin Luther King Jr.    January 15, 1929   
49           James Baldwin     August 02, 1924   
51       Eleanor Roosevelt    October 11, 1884   


In [8]:
t  = pd.read_csv('author_key_file.csv', delimiter=",") 
for col in t.columns:
    print(col)

name
key
gender


### Source 3: Author Location Data
This direct download XML data source contains the location information that will be used
to direct app users to the historical sites associated with each author.  The *id*
tag can be used to match the previously assembled author data (ie, the *key* attribute) with
the author location data.

You are to retain all of the location attributes (excluding *id* - which will be a
duplicate of the existing *key* attribute) from this data source to extend the
previously collected author data.

```
Direct Download URL: https://www.drivehq.com/file/DFPublishFile.aspx/FileID7657515244/Keycqacws4cypvo/author_location_data.xml
Data Format: XML
Encoding: UTF-8
```
#### Tasks
1. Ingest the XML data into a pandas dataframe
2. Use the pandas dataframe merge method to join the new dataframe
with the Source 1+2 dataframe.
3. *Source 3* contains location data for many public figures in addition
to the 50 authors in which we are currently interested. You must restrict your
final Source 3 dataframe to our 50 authors.

### Source 4: Zip Code API
You may have observed that the *Source 3* data does not include the city and zip code information for
the property locations for our 50 authors.  Your task is the use the latitude and longitude
values from the *Source 3* data to locate the city and zip code information and incorporate
that information into your dataframe.

The USPS maintains data related to all US zip codes and their centroid latitude and longitude.  The
*Source 3* data contains the latitude and longitude information (the lat and long tags) for the
author locations. Your task
is to use the *Source 3* latitude and longitude values as an argument to the zip code api in order to
to augment your dataframe with the city name and zip code of the authors' locations.

The ZIP code API contains ZIP codes for the
continental United States, Alaska, Hawaii, Puerto Rico, and American Samoa.
The API provides data in JSON format values for for ZIP code, city, latitude, longitude,
timezone (offset from GMT).

The relevant zip code API URL and parameters are shown in the table below.


API Information and Parameters | Value
---------------------|------
API Documentation | https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-zc-point/api/
API URL | https://public.opendatasoft.com/api/records/1.0/search/
dataset (parameter) | georef-united-states-of-america-zc-point
rows (parameter) | 100
geofilter.distance (parameter) | TBD by You. If used, distance value must by at least 10 kilometers
Geofilter Documentation | https://help.opendatasoft.com/platform/en/exploring_catalog_and_datasets/03_searching_the_data/search.html#geo-filtering
q (parameter) | TBD by You. If used, distance argument must by at least 10 kilometers. **Hint:** use the #distance function.
Search Documentation | https://help.opendatasoft.com/platform/en/exploring_catalog_and_datasets/03_searching_the_data/search.html#full-text-search
**Notes:** | use only one of either the *q* or *geofilter.distance* parameters. Do not use both.  The choice is yours.

#### Tasks
The long and lat values from *Source 3* are the exact longitude and latitude values for a
single property location.  As such, it is unlikely that you will find an exact match from the
zip code API using these values. Instead, you must:

1. round the lat and long values to 2 decimal place
2. invoke the API requesting all zip code related data within 10 kilometers from the
rounded lat and long values. (Refer to the *q* or *geofilter.distance* parameter documentation listed above.)
3. If the returned JSON results indicate more that one candidate record has been returned, you
must determine which record's longitude and latitude are the **closest** to the *Source 3*
long and lat values.
<br/><br/>There are several ways to accomplish this. For example, you could calculate the distance between the
each result's longitude,latitude values and the the long, lat values from *Source 3*. The
distance between the values can be calculated using the Pythagorean Theorem as follows: <br/>
&radic;( (lon<sub>1</sub>-lon<sub>2</sub>)<sup>2</sup> + (lat<sub>1</sub>-lat<sub>2</sub>)<sup>2</sup> )
 <br/><br/> Another, far better, approach involves a careful reading the documentation paying special attention to the *sort*
 parameter. This is a hint...
4. Use the city and zip code that are associated with the result having the
 minimum distance from (closest to) the *Source 3* long, lat values.

## Proof of Concept
Use the dataframe that you constructed above to answer the following three questions.

### Question 1
What is the mean and standard deviation of living space (sqft_living) by number of bedrooms? Use the pandas
dataframe.agg() method to calculate these statistics in a single step.

### Question 2
How many authors, by gender, live on lots that exceed the mean lot size (sqft_lot) by more
than 1 standard deviation.

### Question 3
Examine the documentation for the pandas dataframe.cut() method
[Docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html)

Use this method to convert the *price* attribute into to
3 approximately equal sized bins. Assign each bin specific labels of 'low', 'medium', 'high'.
How many authors are there in each price category?

## Deliverable
Due to the the implementation of the JNB app, it is very easy to create circular variable dependencies. Such
dependencies will thwart my ability to run your entire JNB solution and result in the loss of valuable project points.

To ensure that your have not inadvertently created such dependencies, I **strongly** recommend you perform the following steps
prior to submitting your JNB.

1. Clear all outputs from your JNB solution's code cells
1. Save you JNB solution
1. Stop and restart your JNB kernel via the *Kernel* Jupyter Notebook menu item. Use the
*Restart & Run All Output* menu item.
1. Inspect your output cells for any errors

Once you have completed your project, upload the JNB containing your solution to the
*Course Project* assessment item located in the *Course Project* content area on our Blackboard site.